In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
import os
sys.path.append('..')

import torch
import optuna
import pytorch_lightning as pl

from radianet.datasets import SampleDataset3D
from radianet.config import Config
from radianet.models import Simple3DCNN
from radianet.callbacks import MetricsCallback
from radianet.lightningnet import LightningNet

fdsfd


In [4]:
config = Config

def objective(trial):
    
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        os.path.join(config.MODEL_DIR, "trial_{}".format(trial.number), "{epoch}"), 
        monitor="val_loss", save_top_k = -1
    )
    metrics_callback = MetricsCallback()

    trainer = pl.Trainer(
        logger=False,
        max_epochs=config.EPOCHS,
        checkpoint_callback=checkpoint_callback,
        callbacks=[metrics_callback],
        gpus=0 if torch.cuda.is_available() else None
    )

    n_sample = 5
    datasets = SampleDataset3D(n_sample = n_sample)
    datasets = datasets.get_dataset()

    lightning_model = LightningNet(trial, config, Simple3DCNN, datasets)
    trainer.fit(lightning_model)

    return metrics_callback.metrics[-1]["val_loss"]

study = optuna.create_study(study_name = config.NAME, storage = config.DB_NAME, load_if_exists = True)
study.optimize(objective, n_trials= config.N_TRIALS)


[I 2020-05-09 00:06:37,328] Using an existing study with name 'test' instead of creating a new one.
INFO:lightning:GPU available: True, used: False
INFO:lightning:
   | Name                      | Type        | Params
------------------------------------------------------
0  | model                     | Simple3DCNN | 2 M   
1  | model.conv_block          | Sequential  | 300   
2  | model.conv_block.0        | Conv3d      | 280   
3  | model.conv_block.1        | BatchNorm3d | 20    
4  | model.conv_block.2        | ReLU        | 0     
5  | model.conv_block.3        | MaxPool3d   | 0     
6  | model.conv_block.4        | Dropout     | 0     
7  | model.feedforward_block   | Sequential  | 2 M   
8  | model.feedforward_block.0 | Linear      | 2 M   
9  | model.feedforward_block.1 | ReLU        | 0     
10 | model.feedforward_block.2 | Dropout     | 0     
11 | model.feedforward_block.3 | Linear      | 43    
12 | model.feedforward_block.4 | Sigmoid     | 0     


..\radianet\lightningnet.py:48: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])) is deprecated. Please ensure they have the same size.
  return {"loss": F.binary_cross_entropy(output, target)}


INFO:lightning:Detected KeyboardInterrupt, attempting graceful shutdown...


KeyboardInterrupt: 

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))